**Audio vs. Audio-Visual Model**

The code below trains a Audio vs. Audio-visual classfier model on all the 16 subject data. There are four different RNN models and a Gaussian Bayes model. 

In [1]:
#use these lines to mount google drive for free access to google colab gpu
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mne

     |████████████████████████████████| 6.8MB 5.4MB/s 


In [3]:
#import packages
import mne # For loading EEG data
import numpy as np
#from praatio import tgio # For importing textgrids with word level transcriptions
import glob 
import os

from sklearn import preprocessing
from matplotlib import pyplot as plt
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
%matplotlib inline
from sklearn.metrics import accuracy_score

# Classifier code


Load individual subject data after pre-processing using TIMIT-MovieTrailer-AV-model-data-loader.ipynb.



In [4]:
# choose 50 random indexes as test data from total 373 timit sentences. This set is limited to 373 because, for one subject, only 373 recording were present out of 380. 
import random
test_list = random.sample(range(373), 115)
train_list = [i for i in range(373) if i not in test_list]
print(len(test_list), len(train_list))

115 258


In [13]:
#concatenate timit and movie files based on test and train subject split 
data_dir = '/content/drive/My Drive'
train = ['MT0001','MT0002', 'MT0003','MT0004','MT0005','MT0006', 'MT0008','MT0009','MT0010', 'MT0011', 'MT0012', 'MT0013', 'MT0014', 'MT0015', 'MT0016', 'MT0017']

for num,subject in enumerate(train):
    timit_filename = data_dir + '/EEG_classifier/'+ subject + 'timit.npy'
    movie_filename = data_dir + '/EEG_classifier/'+ subject + 'movie.npy'
    print(num,subject)
    if num == 0:
        timit = np.load(timit_filename)
        movie = np.load(movie_filename)
        print(timit.shape)
        train_timit = timit[train_list,:,:]
        train_movie = movie[train_list,:,:]

        test_timit = timit[test_list,:,:]
        test_movie = movie[test_list,:,:]
    else:
        timit = np.load(timit_filename)
        print(timit.shape)
        train_timit = np.concatenate((train_timit, timit[train_list,:,:]), axis = 0)
        movie = np.load(movie_filename)
        train_movie = np.concatenate((train_movie, movie[train_list,:,:]), axis = 0)

        test_timit = np.concatenate((test_timit, timit[test_list,:,:]), axis = 0)
        test_movie = np.concatenate((test_movie, movie[test_list,:,:]), axis = 0)



0 MT0001
(380, 64, 350)
1 MT0002
(380, 64, 350)
2 MT0003
(373, 64, 350)
3 MT0004
(380, 64, 350)
4 MT0005
(379, 64, 350)
5 MT0006
(379, 64, 350)
6 MT0008
(380, 64, 350)
7 MT0009
(379, 64, 350)
8 MT0010
(380, 64, 350)
9 MT0011
(380, 64, 350)
10 MT0012
(378, 64, 350)
11 MT0013
(380, 64, 350)
12 MT0014
(380, 64, 350)
13 MT0015
(379, 64, 350)
14 MT0016
(379, 64, 350)
15 MT0017
(378, 64, 350)


In [14]:
train_timit = np.reshape(train_timit,(train_timit.shape[0],train_timit.shape[2],train_timit.shape[1]))
train_movie = np.reshape(train_movie,(train_movie.shape[0],train_movie.shape[2],train_movie.shape[1]))
print(train_timit.shape, train_movie.shape)


#y_train and y_test for timit and movie trailer
train_shape = np.shape(train_timit)[0]
y_train_timit = np.ones(train_shape)

train_shape = np.shape(train_movie)[0]
y_train_movie = np.zeros(train_shape)

print(y_train_timit.shape)
print(y_train_movie.shape)



test_timit = np.reshape(test_timit,(test_timit.shape[0],test_timit.shape[2],test_timit.shape[1]))
test_movie = np.reshape(test_movie,(test_movie.shape[0],test_movie.shape[2],test_movie.shape[1]))

test_shape = np.shape(test_timit)[0]
y_test_timit = np.ones(test_shape)

test_shape = np.shape(test_movie)[0]
y_test_movie = np.zeros(test_shape)
print(test_timit.shape, test_movie.shape)
print(y_test_timit.shape)
print(y_test_movie.shape)


(4128, 350, 64) (4128, 350, 64)
(4128,)
(4128,)
(1840, 350, 64) (1840, 350, 64)
(1840,)
(1840,)


In [15]:
#concatenate timit and movie EEG data


train_X = np.concatenate((train_timit, train_movie), axis = 0)
test_X = np.concatenate((test_timit, test_movie), axis = 0)

train_y = np.concatenate((y_train_timit, y_train_movie))
test_y = np.concatenate((y_test_timit, y_test_movie))

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(8256, 350, 64) (8256,) (3680, 350, 64) (3680,)


In [16]:
#calculate % of test data
print(test_X.shape[0]/(test_X.shape[0]+train_X.shape[0]))

0.30831099195710454


In [17]:
#PCA the train and test data

pca_case = False

train_1 = train_X.shape[0]
test_1 = test_X.shape[0]
train_X = np.reshape(train_X, ((train_X.shape[0]*350,64)))
test_X = np.reshape(test_X, ((test_X.shape[0]*350,64)))


standard = StandardScaler()
train_X = standard.fit_transform(train_X)
test_X = standard.transform(test_X)

n_components = 64
if pca_case:
  pca = decomposition.PCA(n_components= n_components)
  pca.fit(train_X)
  train_X_pca = pca.transform(train_X)
  test_X_pca = pca.transform(test_X)


  #reshape data into 3-D on PCA data
  train_X_pca = np.reshape(train_X_pca, ((train_1,350,n_components)))
  test_X_pca = np.reshape(test_X_pca, ((test_1,350,n_components)))

else:
  #reshape data into 3-D on non-pca data
  train_X = np.reshape(train_X, ((train_1,350,n_components)))
  test_X = np.reshape(test_X, ((test_1,350,n_components)))

In [18]:
#Print the distribution of 1s and 0s in train and test
from collections import Counter
print(Counter(test_y).keys()) 
print(Counter(test_y).values()) 

print(Counter(train_y).keys()) 
print(Counter(train_y).values())

dict_keys([1.0, 0.0])
dict_values([1840, 1840])
dict_keys([1.0, 0.0])
dict_values([4128, 4128])


In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU, Dropout, BatchNormalization, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In the below sections, we train 4 RNN models and Bayes model

In [70]:
## final experiments - pca - gru

model = Sequential()
model.add(GRU(100, input_shape=(train_X_pca.shape[1],train_X_pca.shape[2]), return_sequences=True))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(GRU(100, return_sequences=True))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(GRU(100, return_sequences=True))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(GRU(50))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.build()
print(model.summary())
model.fit(train_X_pca, train_y, epochs=30, batch_size=64, verbose = 1)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_4 (GRU)                  (None, 350, 100)          49800     
_________________________________________________________________
batch_normalization_6 (Batch (None, 350, 100)          400       
_________________________________________________________________
activation_6 (Activation)    (None, 350, 100)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 350, 100)          0         
_________________________________________________________________
gru_5 (GRU)                  (None, 350, 100)          60600     
_________________________________________________________________
batch_normalization_7 (Batch (None, 350, 100)          400       
_________________________________________________________________
activation_7 (Activation)    (None, 350, 100)         

In [71]:
# Final evaluation of the model
scores = model.evaluate(test_X_pca, test_y, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 97.55%


In [72]:
## final experiments - pca - LSTM

model = Sequential()
model.add(LSTM(100, input_shape=(train_X_pca.shape[1],train_X_pca.shape[2]), return_sequences=True))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(LSTM(100, return_sequences=True))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(LSTM(100, return_sequences=True))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(LSTM(50))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.build()
print(model.summary())
model.fit(train_X_pca, train_y, epochs=30, batch_size=64, verbose = 1)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 350, 100)          66000     
_________________________________________________________________
batch_normalization_9 (Batch (None, 350, 100)          400       
_________________________________________________________________
activation_9 (Activation)    (None, 350, 100)          0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 350, 100)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 350, 100)          80400     
_________________________________________________________________
batch_normalization_10 (Batc (None, 350, 100)          400       
_________________________________________________________________
activation_10 (Activation)   (None, 350, 100)         

In [73]:
# Final evaluation of the model
scores = model.evaluate(test_X_pca, test_y, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 95.52%


In [58]:
## final experiments - no pca - gru

model = Sequential()
model.add(GRU(100, input_shape=(train_X.shape[1],train_X.shape[2]), return_sequences=True))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(GRU(100, return_sequences=True))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(GRU(100, return_sequences=True))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(GRU(50))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.build()
print(model.summary())
model.fit(train_X, train_y, epochs=30, batch_size=64, verbose = 1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 350, 100)          49800     
_________________________________________________________________
batch_normalization (BatchNo (None, 350, 100)          400       
_________________________________________________________________
activation (Activation)      (None, 350, 100)          0         
_________________________________________________________________
dropout (Dropout)            (None, 350, 100)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 350, 100)          60600     
_________________________________________________________________
batch_normalization_1 (Batch (None, 350, 100)          400       
_________________________________________________________________
activation_1 (Activation)    (None, 350, 100)          0

In [59]:
# Final evaluation of the model
scores = model.evaluate(test_X, test_y, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 98.15%


In [60]:
## final experiments - no pca  - LSTM

model = Sequential()
model.add(LSTM(100, input_shape=(train_X.shape[1],train_X.shape[2]), return_sequences=True))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(LSTM(100, return_sequences=True))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(LSTM(100, return_sequences=True))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(LSTM(50))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.build()
print(model.summary())
model.fit(train_X, train_y, epochs=30, batch_size=64, verbose = 1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 350, 100)          66000     
_________________________________________________________________
batch_normalization_3 (Batch (None, 350, 100)          400       
_________________________________________________________________
activation_3 (Activation)    (None, 350, 100)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 350, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 350, 100)          80400     
_________________________________________________________________
batch_normalization_4 (Batch (None, 350, 100)          400       
_________________________________________________________________
activation_4 (Activation)    (None, 350, 100)         

In [61]:
# Final evaluation of the model
scores = model.evaluate(test_X, test_y, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 94.48%


**Bayes Model**

In [20]:
from sklearn.naive_bayes import GaussianNB

#reshape into 2-D data
train_X = np.reshape(train_X, ((train_X.shape[0]*350,64)))
test_X = np.reshape(test_X, ((test_X.shape[0]*350,64)))

#naive bayes filling the label
train_l_ = np.zeros((train_X.shape[0]))
test_l_ = np.zeros((test_X.shape[0]))

for i in range(train_y.shape[0]):
  train_l_[i*350:i*350+350] = train_y[i]

for i in range(test_y.shape[0]):
  test_l_[i*350:i*350+350] = test_y[i]


#without pca - Gaussian Naive Bayes model
gnb = GaussianNB()
y_pred = gnb.fit(train_X, train_l_).predict(test_X)
accuracy_score(y_pred, test_l_)

0.49060326086956524